In [ ]:
from availableTools import *
from langfuse import Langfuse
from langfuse.openai import OpenAI
from langfuse.decorators import observe
import json
from Utils.jinjaProcessor import *
from Utils.parser import *
from Utils.rag import *
import requests
from dotenv import load_dotenv
load_dotenv(override=True)
langfuse = Langfuse()

In [ ]:
class Chatbot:
    def __init__(self):
        self.api_keys = []
        self.current_index = 0
        self.insert_api_key()
        self.tools = process_template_no_var('Prompt/tools_template.jinja')
        self.functions = {
            "getProductList": getProductList,
            "getCurrentCart": getCurrentCart,
            "addProduct": addProduct,
            "modifyCart": modifyCart,
            "removeProduct": removeProduct,
            "greetings": greetings,
        }
        try:
            self.dbcollection = createDbCollection(
                dbname="IndoJuni",
                filename="Knowledge base/productCatalog.json",
                dbpath="VectorDB"
            )
        except Exception as e:
            self.dbcollection = getDbCollection(
                dbpath="VectorDB",
                dbname="IndoJuni"
            )

    def insert_api_key(self):
        with open("llm_api_keys.txt") as f:
            for line in f.readlines():
                self.api_keys.append(line.strip())
    
    def get_client(self,index):
        return OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key= self.api_keys[index]
        )
    
    @observe()
    def generate_response(self,messages,current_index):
        try:
            client = self.get_client(current_index)
            response = client.chat.completions.create(
                model = "llama-3.3-70b-versatile",
                messages = messages,
                temperature=0.1,
                top_p=0.1,
                presence_penalty=0.0,
                frequency_penalty=0.0,
            )
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                current_index = (current_index + 1) % len(self.api_keys)
                client = self.get_client(current_index)
                response = client.chat.completions.create(
                    model = "llama-3.3-70b-versatile",
                    messages = messages,
                    temperature=0.1,
                    top_p=0.1,
                    presence_penalty=0.0,
                    frequency_penalty=0.0,
                )
        return response

    def end_response(self,tool_call):
        end_response_func = ["NONE"]
        if tool_call['function_name'] in end_response_func:
            return True
        else:
            return False
    
    def generate_single_chat_message(self,user_prompt,messages,flag):
    
        chat_message = []

        self.dbcollection = getDbCollection(dbpath="Data/Chroma databases/Applestore",dbname="Applestore")

        context = RAG(dbcollection=self.dbcollection,
                    query=user_prompt)

        temp = {
            "tools": self.tools,
            "context": context
        }

        system_prompt = process_template('Prompt/system_prompt.jinja', temp)

        if flag == False:
            messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]
            flag = True
        else:
            messages.append({
                "role": "user",
                "content": user_prompt
            })
            messages[0]['content'] = system_prompt

        while True:
            response = self.generate_response(messages)
            messages.append({
                "role": "assistant",
                "content": response
            })

            
            tools = parse_function(response,bfcl_format=False)

            if len(tools) == 1 and (tools[0]['name'] == 'FUNCTION_NOT_FOUND' or tools[0]['name'] == 'NONE'):
                break
            
            
            for tool in tools:

                if tool['name'] == 'FUNCTION_NOT_FOUND' or tool['name'] == 'NONE':
                    continue

                # Check for function name
                try:
                    function_name = self.functions[tool['name']]
                except:
                    messages.append({
                        "role": "tool",
                        "tool_call_id": "N/A",
                        "content": "Function not found in tools_dict"
                    })

                    continue

                try:
                    function_args = tool['args']
                except:
                    function_args = tool['parameters']

                if "<MISSING>" in function_args.values():
                    return messages, flag, context, chat_message

                # Check for function arguments
                try:    
                    function_output = function_name(**function_args)
                    content = json.dumps(function_output, indent=4)
                    messages.append({
                        "role": "tool",
                        "tool_call_id": function_output['tool_call_id'],
                        "content": content
                    })

                except Exception as e:
                    messages.append({
                        "role": "tool",
                        "tool_call_id": "N/A",
                        "content": f"Error: {str(e)} calling {function_name.__name__} with args {function_args}"
                    })

        return messages, flag, context, chat_message
    
    def run_conversation(self):
        messages = []
        flag = False

        count = 0
        while True:
            
            tester_message = input("Tester Message: ")
            if tester_message.strip().lower() == 'exit':
                print("Exiting the chatbot.")
                break
                
            messages, flag, context, chat_message = self.generate_single_chat_message(tester_message, messages,flag)

            count += 1
